# Input Data from WRDS Sources

In [1]:
#%% Packages 
import wrds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 6]

import time
from time import strptime, strftime

# Setups
pd.set_option("display.max_rows", 200) # max number or rows to be displayed 
#%% Set WRDS Connection
db = wrds.Connection(wrds_username='zrsong') # make sure to configure wrds connector before hand.


Loading library list...
Done


In [2]:
# Import the required datasets
sp500_returns = db.get_table(library='crsp_a_indexes', table='msp500', columns=['caldt', 'vwretd', 'vwretx'])

# Close the database connection
db.close()

# collapse data down to annually
sp500_returns['caldt'] = pd.to_datetime(sp500_returns['caldt'])
sp500_returns['year'] = sp500_returns['caldt'].dt.year
sp500_returns = sp500_returns.groupby('year').agg({'vwretd': 'sum', 'vwretx': 'sum'}).reset_index()
sp500_returns = sp500_returns.groupby('year').agg({'vwretd': 'sum', 'vwretx': 'sum'}).reset_index()


In [3]:
# Read the data from the file
tfz = pd.read_csv('../Data/tfz_mth_ft_data.csv')

# Keep only the rows where kytreasnox is equal to 2000002 and 2000007
tfz_filtered = tfz[tfz['kytreasnox'].isin([2000007])].copy()

# Aggregate the data for the two kytreasnox keys at the annual level
tfz_filtered['mcaldt'] = pd.to_datetime(tfz_filtered['mcaldt'])
tfz_filtered['year'] = tfz_filtered['mcaldt'].dt.year
tfz_aggregated = tfz_filtered.groupby(['year', 'kytreasnox']).agg({'tmretadj': 'sum', 'tmytm': 'mean'}).reset_index()
# divide by 100 to convert the data back to percentage
tfz_aggregated['tmretadj'] = tfz_aggregated['tmretadj'] / 100
tfz_aggregated['tmytm'] = tfz_aggregated['tmytm'] / 100

# Display the aggregated data
print(tfz_aggregated)

    year  kytreasnox  tmretadj     tmytm
0   1941   2000007.0  0.013498  0.019210
1   1942   2000007.0  0.021997  0.020623
2   1943   2000007.0  0.024564  0.019760
3   1944   2000007.0  0.026520  0.019401
4   1945   2000007.0  0.056504  0.015429
5   1946   2000007.0  0.007507  0.014368
6   1947   2000007.0 -0.017684  0.016876
7   1948   2000007.0  0.032884  0.020676
8   1949   2000007.0  0.062786  0.021819
9   1950   2000007.0  0.000665  0.021622
10  1951   2000007.0 -0.039628  0.024739
11  1952   2000007.0  0.011970  0.026453
12  1953   2000007.0  0.036880  0.029016
13  1954   2000007.0  0.047932  0.025059
14  1955   2000007.0 -0.014928  0.027996
15  1956   2000007.0 -0.049368  0.030575
16  1957   2000007.0  0.094348  0.034077
17  1958   2000007.0 -0.053726  0.032185
18  1959   2000007.0 -0.045025  0.041561
19  1960   2000007.0  0.137521  0.039186
20  1961   2000007.0  0.000183  0.038505
21  1962   2000007.0  0.084897  0.038583
22  1963   2000007.0  0.019792  0.039609
23  1964   20000

In [4]:
# read in rf data and convert variables names to lowercase
rf = pd.read_csv('../Data/rf_series.csv')
rf.columns = rf.columns.str.lower()

# Keep only the rows where kytreasnox is equal to 2000002
rf_filtered = rf[rf['kytreasnox'].isin([2000002])].copy()

# Aggregate the data for the kytreasnox key at the annual level
rf_filtered['mcaldt'] = pd.to_datetime(rf_filtered['mcaldt'])
rf_filtered['year'] = rf_filtered['mcaldt'].dt.year
rf_aggregated = rf_filtered.groupby(['year', 'kytreasnox']).agg({'tmytm': 'mean'}).reset_index()
# divide by 100 to convert the data back to percentage
rf_aggregated['tmytm'] = rf_aggregated['tmytm'] / 100

In [14]:
# read in AAA index
AAA = pd.read_csv('../Data/AAA_fred.csv')
# generate year
AAA['year'] = pd.to_datetime(AAA['DATE']).dt.year

# convert AAA index to numeric and change to percentage
AAA['AAA'] = pd.to_numeric(AAA['AAA'], errors='coerce') / 100


In [ ]:
# merge the rf, tfz and sp500 data 
#data = pd.merge(tfz_aggregated, rf_aggregated, on='year', suffixes=('_tfz', '_rf'))
#data = pd.merge(data, sp500_returns, left_on='year', right_on='year', suffixes=('_tfz', '_sp500'))

# keep only year, tmretadj, tmytm_rf and vwretd
#data = data[['year', 'tmytm_tfz', 'tmytm_rf', 'vwretd']]
#data.columns = ['year', '10_yr_treasury', '3_month_tbill', 'sp500_return']
# keep year >= 1960
#data = data[data['year'] >= 1960]


In [18]:
# Merge sp500_returns and AAA on 'year'
data = pd.merge(sp500_returns, AAA, on='year')

# Merge the result with rf_aggregated on 'year'
data = pd.merge(data, rf_aggregated, on='year')

# keep only year, vwretd, AAA, tmytm_x, tmytm
data = data[['year', 'vwretd', 'AAA', 'tmytm']]
# rename columns
data.columns = ['year', 'sp500', 'AAA', 'rf']

# keep year >= 1960
data = data[data['year'] >= 1960]

# Ask ChatGPT to Use the Data to Form Portfolios using the prompt below

Use the S&P500 returns and 10-year treasury bond and 3-month treasury bill yield to calculate annual portfolio value for the below strategy. Start with $1 million each year from 1960 to 2003. Invest 90% of it in S&P500, and 10% in a money market fund (3-month t-bill). Reinvest cash dividends into the S&P 500 portfolio, and interest on money market into the money market each year. Rebalance the portfolio annually to bring 90% investment in S&P500 and 10% in money market. Calculate the value of such a portfolio at the end of 20 years from initial investment of $1 million using actual annual returns on S&P500 and money market funds. The strategy is implemented each start year with an investment of $1 million from 1960 to 2003, i.e., a total of 44 start years. Calculate mean, median, max, min, and standard deviation for the distribution of 44 portfolio values on implementing this investment strategy.

In [20]:
# define start years from 1960 to 2003
start_years = list(range(1960, 2003))

def calculate_portfolio_values(num_years):
    portfolio_values = []

    # Iterate over start years
    for start_year in start_years:
        # Initialize portfolio value
        portfolio_value = 1000000

        # Iterate over num_years
        for year in range(start_year, start_year + num_years):
            # Calculate portfolio allocation
            sp500_allocation = portfolio_value * 0.9
            money_market_allocation = portfolio_value * 0.1

            # Calculate portfolio returns
            sp500_return = float(data[data['year'] == year]['sp500'].values[0])
            money_market_return = float(data[data['year'] == year]['rf'].values[0])

            # Calculate portfolio values at the end of the year
            sp500_value = sp500_allocation * (1 + sp500_return)
            money_market_value = money_market_allocation * (1 + money_market_return)

            # Rebalance portfolio
            total_value = sp500_value + money_market_value
            sp500_allocation = total_value * 0.9
            money_market_allocation = total_value * 0.1

            # Update portfolio value
            portfolio_value = sp500_allocation + money_market_allocation

        # Append portfolio value to the list
        portfolio_values.append(portfolio_value)

    # Create DataFrame of portfolio values
    portfolio_df = pd.DataFrame({'start_year': start_years, 'portfolio_value': portfolio_values})

    # Return statistics and portfolio DataFrame
    return portfolio_df

# Call the function with num_years = 20
start_years = list(range(1960, 2003))
portfolio_20yr = calculate_portfolio_values(20)
start_years = list(range(1960, 2008))
portfolio_15yr = calculate_portfolio_values(15)
start_years = list(range(1960, 2013))
portfolio_10yr = calculate_portfolio_values(10)

# bind the data together horizontally
portfolio_df1 = pd.concat([portfolio_20yr, portfolio_15yr, portfolio_10yr], axis=1)
portfolio_df1.columns = ['start_year_20yr', 'portfolio_value_20yr', 'start_year_15yr', 'portfolio_value_15yr', 'start_year_10yr', 'portfolio_value_10yr']

# Output a table for the mean, medium, min and max portfolio values for 20yr, 10yr, 5yr only
portfolio_df1[['portfolio_value_20yr', 'portfolio_value_15yr', 'portfolio_value_10yr']].describe()

,portfolio_value_20yr,portfolio_value_15yr,portfolio_value_10yr
count,4.300000e+01,4.800000e+01,5.300000e+01
mean,7.404676e+06,4.311468e+06,2.639118e+06
std,4.585765e+06,2.434034e+06,1.111739e+06
min,2.589599e+06,1.684402e+06,8.609713e+05
25%,4.020090e+06,2.371538e+06,1.771197e+06
50%,5.623856e+06,3.497816e+06,2.255364e+06
75%,9.324701e+06,6.134628e+06,3.606512e+06
max,2.035429e+07,1.047568e+07,4.723757e+06


Use the S&P500 returns and 10-year treasury bond and 3-month treasury bill yield to calculate annual portfolio value for the below strategy. Start with $1 million each year from 1960 to 2003. Invest 50% of it in S&P500, 40% in bond market (10-year t-bond), and 10% in a money market fund (3-month t-bill). Reinvest the returns each year into the portfolio. Rebalance the portfolio annually to bring 50% investment in S&P500, 40% in bond market, and 10% in money market. Calculate the value of such a portfolio at the end of 20, 10, 5 years from initial investment of $1 million using actual annual returns on S&P500, bond return, and money market fund returns. The strategy is implemented each start year with an investment of $1 million from 1960 to 2003 or 2013 or 2018, i.e., a total of 44, 54, and 59 start years. Calculate mean, median, max, min, and standard deviation for the distribution of 44 portfolio values on implementing this investment strategy.

In [21]:
def calculate_portfolio_values(num_years):
    # Initialize portfolio values list
    portfolio_values = []

    # Iterate over start years
    for start_year in start_years:
        # Initialize portfolio value
        portfolio_value = 1000000

        # Iterate over years
        for year in range(start_year, start_year + num_years):
            # Calculate portfolio allocations
            sp500_allocation = portfolio_value * 0.5
            bond_allocation = portfolio_value * 0.4
            money_market_allocation = portfolio_value * 0.1

            # Calculate portfolio returns
            sp500_return = float(data[data['year'] == year]['sp500'].values[0])
            bond_return = float(data[data['year'] == year]['AAA'].values[0])
            money_market_return = float(data[data['year'] == year]['rf'].values[0])

            # Calculate portfolio values at the end of the year
            sp500_value = sp500_allocation * (1 + sp500_return)
            bond_value = bond_allocation * (1 + bond_return)
            money_market_value = money_market_allocation * (1 + money_market_return)

            # Rebalance portfolio
            total_value = sp500_value + bond_value + money_market_value
            sp500_allocation = total_value * 0.5
            bond_allocation = total_value * 0.4
            money_market_allocation = total_value * 0.1

            # Update portfolio value
            portfolio_value = total_value

        # Append portfolio value to the list
        portfolio_values.append(portfolio_value)
        
    # Create DataFrame of portfolio values
    portfolio_df = pd.DataFrame({'start_year': start_years, 'portfolio_value': portfolio_values})

    # Return portfolio values
    return portfolio_df

# Call the function with num_years = 20
start_years = list(range(1960, 2003))
portfolio_20yr = calculate_portfolio_values(20)
start_years = list(range(1960, 2008))
portfolio_15yr = calculate_portfolio_values(15)
start_years = list(range(1960, 2013))
portfolio_10yr = calculate_portfolio_values(10)

# bind the data together horizontally
portfolio_df2 = pd.concat([portfolio_20yr, portfolio_15yr, portfolio_10yr], axis=1)
portfolio_df2.columns = ['start_year_20yr', 'portfolio_value_20yr', 'start_year_15yr', 'portfolio_value_15yr', 'start_year_10yr', 'portfolio_value_10yr']

# Output a table for the mean, medium, min and max portfolio values for 20yr, 10yr, 5yr only
portfolio_df2[['portfolio_value_20yr', 'portfolio_value_15yr', 'portfolio_value_10yr']].describe()

,portfolio_value_20yr,portfolio_value_15yr,portfolio_value_10yr
count,4.300000e+01,4.800000e+01,5.300000e+01
mean,6.102059e+06,3.736584e+06,2.367174e+06
std,2.573819e+06,1.449802e+06,6.973257e+05
min,2.768659e+06,1.982060e+06,1.252736e+06
25%,4.019165e+06,2.527489e+06,1.793983e+06
50%,5.838133e+06,3.346099e+06,2.210090e+06
75%,7.393646e+06,5.005034e+06,3.088429e+06
max,1.212720e+07,6.442779e+06,3.763495e+06
